<a href="https://colab.research.google.com/github/LaviBenshimol/Deep-Learning/blob/master/OAI_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Offensive AI Course Homework
* Course website: https://ymirsky.github.io/course/
* Course edition: Fall 2024
* Lecturer: Dr. Yisroel Mirsky
* Due Date: 31/12/24
## Instructions
* This homework can be done in groups of **two**
* Make a copy of this notebook and solve the problems below.
* Do not use the ART toolbox in this homework
* Orgnization: In each section there are two cells: (1) a cell with a function with some missing code where you must complete the missing code (do not change the existing code), (2) a cell to test your code. Before you submit your work, make sure that the test cell in every section can run your code.
* Submission: (1) generate a sharable link to your notebook, (2) submit your link by [clicking here](https://docs.google.com/forms/d/e/1FAIpQLSexY-uV7jTOosgRt863ltfcHTsNO6lWanMXCzQd_uglgddodw/viewform?usp=sf_link)  
* Each day past the deadline is -15 pnts


## Sections:
1. BIM (40 points)
2. Random Starts (10 points)
3. Masking (10 points)
4. Color Limited Attacks (20 points)
5. PGD (20 points)




# Setup

These cells will setup your environment


In [ ]:
#### Load Modules ####

import os
import json
import math
import time
import numpy as np
import scipy.linalg

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Path to the folder where the datasets are/should be downloaded (e.g. MNIST)
DATASET_PATH = "../data"

# Setting the seed
#pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
#torch.backends.cudnn.determinstic = True
#torch.backends.cudnn.benchmark = False

# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

In [ ]:
#### load dataset ####

import urllib.request
from urllib.error import HTTPError
import zipfile
# Github URL where the dataset is stored for this tutorial
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial10/"
# Files to download
pretrained_files = [(DATASET_PATH, "TinyImageNet.zip")]
# Create checkpoint path if it doesn't exist yet
os.makedirs(DATASET_PATH, exist_ok=True)

# For each file, check whether it already exists. If not, try downloading it.
for dir_name, file_name in pretrained_files:
    file_path = os.path.join(dir_name, file_name)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print(f"Downloading {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)
        if file_name.endswith(".zip"):
            print("Unzipping file...")
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(file_path.rsplit("/",1)[0])


#### load and setup victim model (resnet34) ####

# Load CNN architecture pretrained on ImageNet
victim_model = torchvision.models.resnet34(pretrained=True)
victim_model = victim_model.to(device)

# No gradients needed for the network
victim_model.eval()
for p in victim_model.parameters():
    p.requires_grad = False

# Mean and Std from ImageNet
NORM_MEAN = np.array([0.485, 0.456, 0.406])
NORM_STD = np.array([0.229, 0.224, 0.225])
# No resizing and center crop necessary as images are already preprocessed.
plain_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN,
                         std=NORM_STD)
])

# Load dataset and create data loader
imagenet_path = os.path.join(DATASET_PATH, "TinyImageNet/")
assert os.path.isdir(imagenet_path), f"Could not find the ImageNet dataset at expected path \"{imagenet_path}\". "
dataset = torchvision.datasets.ImageFolder(root=imagenet_path, transform=plain_transforms)
data_loader = data.DataLoader(dataset, batch_size=32, shuffle=True, drop_last=False, num_workers=8)

# Load label names to interpret the label numbers 0 to 999
with open(os.path.join(imagenet_path, "label_list.json"), "r") as f:
    label_names = json.load(f)

def get_label_index(lab_str):
    assert lab_str in label_names, f"Label \"{lab_str}\" not found. Check the spelling of the class."
    return label_names.index(lab_str)

In [ ]:
#### Some helper functions to compute and view results ####

def eval_model(victim_model, dataset_loader, img_func=None):
    tp, tp_5, counter = 0., 0., 0.
    for imgs, labels in tqdm(dataset_loader, desc="Validating..."):
        imgs = imgs.to(device)
        labels = labels.to(device)
        if img_func is not None:
            imgs = img_func(imgs, labels)
        with torch.no_grad():
            preds = victim_model(imgs)
        tp += (preds.argmax(dim=-1) == labels).sum()
        tp_5 += (preds.topk(5, dim=-1)[1] == labels[...,None]).any(dim=-1).sum()
        counter += preds.shape[0]
    acc = tp.float().item()/counter
    top5 = tp_5.float().item()/counter
    print(f"Top-1 error: {(100.0 * (1 - acc)):4.2f}%")
    print(f"Top-5 error: {(100.0 * (1 - top5)):4.2f}%")
    return acc, top5

def show_prediction(img, label, pred, K=5, adv_img=None, noise=None):

    if isinstance(img, torch.Tensor):
        # Tensor image to numpy
        img = img.cpu().permute(1, 2, 0).numpy()
        img = (img * NORM_STD[None,None]) + NORM_MEAN[None,None]
        img = np.clip(img, a_min=0.0, a_max=1.0)
        label = label.item()

    # Plot on the left the image with the true label as title.
    # On the right, have a horizontal bar plot with the top k predictions including probabilities
    if noise is None or adv_img is None:
        fig, ax = plt.subplots(1, 2, figsize=(10,2), gridspec_kw={'width_ratios': [1, 1]})
    else:
        fig, ax = plt.subplots(1, 5, figsize=(12,2), gridspec_kw={'width_ratios': [1, 1, 1, 1, 2]})

    ax[0].imshow(img)
    ax[0].set_title(label_names[label])
    ax[0].axis('off')

    if adv_img is not None and noise is not None:
        # Visualize adversarial images
        adv_img = adv_img.cpu().permute(1, 2, 0).numpy()
        adv_img = (adv_img * NORM_STD[None,None]) + NORM_MEAN[None,None]
        adv_img = np.clip(adv_img, a_min=0.0, a_max=1.0)
        ax[1].imshow(adv_img)
        ax[1].set_title('Adversarial')
        ax[1].axis('off')
        # Visualize noise
        noise = noise.cpu().permute(1, 2, 0).numpy()
        noise = noise * 0.5 + 0.5 # Scale between 0 to 1
        ax[2].imshow(noise)
        ax[2].set_title('Noise')
        ax[2].axis('off')
        # buffer
        ax[3].axis('off')

    if abs(pred.sum().item() - 1.0) > 1e-4:
        pred = torch.softmax(pred, dim=-1)
    topk_vals, topk_idx = pred.topk(K, dim=-1)
    topk_vals, topk_idx = topk_vals.cpu().numpy(), topk_idx.cpu().numpy()
    ax[-1].barh(np.arange(K), topk_vals*100.0, align='center', color=["C0" if topk_idx[i]!=label else "C2" for i in range(K)])
    ax[-1].set_yticks(np.arange(K))
    ax[-1].set_yticklabels([label_names[c] for c in topk_idx])
    ax[-1].invert_yaxis()
    ax[-1].set_xlabel('Confidence')
    ax[-1].set_title('Predictions')

    plt.show()
    plt.close()

Let's make sure the victim model is setup correctly

In [ ]:
exmp_batch, label_batch = next(iter(data_loader))
with torch.no_grad():
    preds = victim_model(exmp_batch.to(device))
for i in range(0,5):
    show_prediction(exmp_batch[i], label_batch[i], preds[i])

# 1. sign-PGD

In this section, complete the code below to impliment the iterative sign-PGD algorithm, for an $\ell_{\infty}$ bound.

Features:
* The user should be able to choose if the attack is targeted or not targeted
* If ``targeted==True``, then sign-PGD should perform a targeted attack using ``labels`` as the target labels. The labels should be class indexes, e.g., ``labels=tensor([23, 4, 7])``
* If ``targeted==False``, then ``labels`` should contain the actual labels for the batch and sign-PGD should perform an untargeted attack.

Reminders:
* alpha is the gradient step size to be applied to ``adv_images``
* eps is the bound of how far ``adv_images`` is allowed to travel from ``images`` (individually)



In [ ]:
def signPGD(model, images, labels, eps=0.3, alpha=0.01, steps=10, targeted=False,  pixelclip=(-2.6,2.6)):

    images = images.clone().to(device)
    labels = labels.clone().to(device)

    loss = nn.CrossEntropyLoss()

    ### Change or move code from here on ###

    adv_images = images.clone().detach()
    adv_images.requires_grad = True
    outputs = model(adv_images)

    # hints:

      # Calculate loss
      # cost = loss(...)

      # Update adversarial images
      # grad = ...

      # clipping...
          # ...
          # the images are zscore normalized so the pixels should not exceed the range in pixelclip

    ### Don't change this code:

    return adv_images, alpha*grad.sign()  # grad is the gradient (pertubation)

In [ ]:
# Untargeted attacks
adv_imgs_ut, noise_grad_ut = signPGD(victim_model, exmp_batch, label_batch, eps=0.3, alpha=.01, steps=10, targeted=False,  pixelclip=(-2.6,2.6))
with torch.no_grad():
    adv_preds_ut = victim_model(adv_imgs_ut.to(device))

# Targeted attacks
targets = (label_batch + 100) % 1000
adv_imgs_t, noise_grad_t = signPGD(victim_model, exmp_batch, targets, eps=0.3, alpha=.01, steps=10, targeted=True,  pixelclip=(-2.6,2.6))
with torch.no_grad():
    adv_preds_t = victim_model(adv_imgs_t.to(device))

print("Untargeted Results")
for i in range(0,5):
    show_prediction(exmp_batch[i], label_batch[i], adv_preds_ut[i], adv_img=adv_imgs_ut[i], noise=100*noise_grad_ut[i])
print("Targeted Results")
for i in range(0,5):
    show_prediction(exmp_batch[i], label_batch[i], adv_preds_t[i], adv_img=adv_imgs_t[i], noise=100*noise_grad_t[i])

# 2. Random Starts

Now let's add random starts to sign-PGD.
A random start is where we start sign-PGD near x in an attempt to find a better solution. This is accomplished by using x + n in the first iteration, where n contains random values and where x + n is within the distance epsilon from x

Feature to impliment:
* Let the user choose whether the algorithm should use a random start or not

Copy your code below to the function ``signPGD2`` and add the option for a random start

In [ ]:
def signPGD2(model, images, labels, eps=0.3, alpha=0.01, steps=10, targeted=False, random_start=True, pixelclip=(-2.6,2.6)):

    # ....

    return adv_images, alpha*grad.sign()

In [ ]:
# let try several trials of BIM on the same image
preds = []
for i in range(40):
  adv_imgs, noise_grad = signPGD2(victim_model, exmp_batch[0:1], label_batch[0:1], eps=0.3, alpha=.01, steps=50, targeted=False, random_start=True, pixelclip=(-2.6,2.6))
  with torch.no_grad():
    pred = victim_model(adv_imgs.to(device))[0,label_batch[0]].cpu().detach().numpy()
    preds.append( pred )

plt.hist(preds)
plt.title("Confidence Distribution Among Attacks on same Image. Lower is better")

# 3. Masked Attacks

In some cases an attack is limited to a certain region within the image. In this section, upgrade your ``signPGD2`` procedure so that it will only add a pertubation to one half of an image, yet still fool the victim.

Features:
* Let the user choose whether to add a pertubation to the left half, right half, or entire image.

Hint: you can multiply a tensor with a tensor of zeroes and ones to erase (focus) on relevant pixels.

In [ ]:
def signPGD3(model, images, labels, eps=0.3, alpha=0.01, steps=10, targeted=False, random_start=True, mask=None, pixelclip=(-2.6,2.6)):
    if mask is None:
      M = torch.ones(images.shape[1:])
    if mask == "left":
      M = torch.zeros(images.shape)
      M[:, :, :, :images.shape[3]//2] = 1
    if mask == "right":
      M = torch.zeros(images.shape)
      M[:, :, :, images.shape[3]//2:] = 1

    #...

    return adv_images, alpha*grad.sign()

In [ ]:
# Untargeted attacks
adv_imgs_ut, noise_grad_ut = signPGD3(victim_model, exmp_batch, label_batch, eps=0.3, alpha=.01, steps=10, targeted=False, mask="left", pixelclip=(-2.6,2.6))
with torch.no_grad():
    adv_preds_ut = victim_model(adv_imgs_ut.to(device))

# Targeted attacks
targets = (label_batch + 100) % 1000
adv_imgs_t, noise_grad_t = signPGD3(victim_model, exmp_batch, targets, eps=0.3, alpha=.01, steps=10, targeted=True, mask="left", pixelclip=(-2.6,2.6))
with torch.no_grad():
    adv_preds_t = victim_model(adv_imgs_t.to(device))

print("Untargeted Results")
for i in range(0,5):
    show_prediction(exmp_batch[i], label_batch[i], adv_preds_ut[i], adv_img=adv_imgs_ut[i], noise=100*noise_grad_ut[i])
print("Targeted Results")
for i in range(0,5):
    show_prediction(exmp_batch[i], label_batch[i], adv_preds_t[i], adv_img=adv_imgs_t[i], noise=100*noise_grad_t[i])

# 4. Color Limited Attacks

Now we want to change a specific color channel of the image (e.g., attack the victim by making the image more red in certain pixels)

Features:
* Let the user choose which channels the pertubation should modify: red, green, blue, or all together (default).

In [ ]:
def signPGD4(model, images, labels, eps=0.3, alpha=0.01, steps=10, targeted=False, random_start=True, mask=None, channel=None, pixelclip=(-2.6,2.6)):
    if mask is None:
      M = torch.ones(images.shape[1:])
    if mask == "left":
      M = torch.ones(images.shape[1:])
      M[:, :, :, :images.shape[3]//2] = 1
    if mask == "right":
      M = torch.ones(images.shape[1:])
      M[:, :, :, images.shape[3]//2:] = 1

    #...if channel...

    return adv_images, alpha*grad.sign()

In [ ]:
# Untargeted attacks
adv_imgs_ut, noise_grad_ut = signPGD4(victim_model, exmp_batch, label_batch, eps=0.3, alpha=.01, steps=10, targeted=False, mask=None, channel="red",  pixelclip=(-2.6,2.6))
with torch.no_grad():
    adv_preds_ut = victim_model(adv_imgs_ut.to(device))

# Targeted attacks
targets = (label_batch + 100) % 1000
adv_imgs_t, noise_grad_t = signPGD4(victim_model, exmp_batch, targets, eps=0.3, alpha=.01, steps=10, targeted=True, mask=None, channel= "red", pixelclip=(-2.6,2.6))
with torch.no_grad():
    adv_preds_t = victim_model(adv_imgs_t.to(device))

print("Untargeted Results")
for i in range(0,5):
    show_prediction(exmp_batch[i], label_batch[i], adv_preds_ut[i], adv_img=adv_imgs_ut[i], noise=100*noise_grad_ut[i])
print("Targeted Results")
for i in range(0,5):
    show_prediction(exmp_batch[i], label_batch[i], adv_preds_t[i], adv_img=adv_imgs_t[i], noise=100*noise_grad_t[i])

# 5. PGD $\ell_2$ (10 points)
Update signPGD4 to perform full gradient (not sign) PGD, and:
- epsilon bounded to an $\ell_2$
- normalized steps
- optimization performed using the ADAM optimizer


In [ ]:
def PGD_l2(model, images, labels, eps=0.3, alpha=0.01, steps=10, targeted=False, random_start=True, mask=None, channel=None, pixelclip=(-2.6,2.6)):
    #...

    return adv_images, alpha*grad.sign()

In [ ]:
# Untargeted attacks
adv_imgs_ut, noise_grad_ut = PGD_l2(victim_model, exmp_batch, label_batch, eps=0.3, alpha=.01, steps=10, targeted=False, mask=None, channel=None,  pixelclip=(-2.6,2.6))
with torch.no_grad():
    adv_preds_ut = victim_model(adv_imgs_ut.to(device))

# Targeted attacks
targets = (label_batch + 100) % 1000
adv_imgs_t, noise_grad_t = PGD_l2(victim_model, exmp_batch, targets, eps=0.3, alpha=.01, steps=10, targeted=True, mask=None, channel= None, pixelclip=(-2.6,2.6))
with torch.no_grad():
    adv_preds_t = victim_model(adv_imgs_t.to(device))

print("Untargeted Results")
for i in range(0,5):
    show_prediction(exmp_batch[i], label_batch[i], adv_preds_ut[i], adv_img=adv_imgs_ut[i], noise=100*noise_grad_ut[i])
print("Targeted Results")
for i in range(0,5):
    show_prediction(exmp_batch[i], label_batch[i], adv_preds_t[i], adv_img=adv_imgs_t[i], noise=100*noise_grad_t[i])